In [1]:
import os 

In [2]:
%pwd

'd:\\pythonProjects\\MLflow_ML_project\\research'

In [3]:
os.chdir('../')
%pwd 

'd:\\pythonProjects\\MLflow_ML_project'

In [4]:
# entity 
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    repo_owner: str 
    repo_name : str 


In [8]:
# config 
from mlProject.constants import * 
from mlProject.utils.common import read_yaml, create_directories, save_json
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH,
            schema_filepath= SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evalution
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        mlflow_tracking=self.config.mlflow_tracking
        create_directories([config.root_dir])

        model_evalution_config=ModelEvaluationConfig(
            root_dir=config.root_dir, 
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name, 
            repo_owner= mlflow_tracking.repo_owner, 
            repo_name = mlflow_tracking.repo_name
        )

        return model_evalution_config



In [16]:
# components 
import os 
import pandas as pd 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub 

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_squared_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2 
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        dagshub.init(repo_name=self.config.repo_name, repo_owner=self.config.repo_owner, mlflow=True)
        tracking_uri_type_store=urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)
            (rmse,mae,r2)=self.eval_metrics(test_y,predicted_qualities)

            #saving metrics as local 
            scores= {'rmse': rmse,'r2_score':r2,'mae':mae}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)
            
            if tracking_uri_type_store!='file':
                # register the model
                mlflow.sklearn.log_model(model,'model',registered_model_name='ElasticnetModel')
            else:
                mlflow.sklearn.log_model(model,'model')




In [18]:
# pipeline 
try:
    config=ConfigurationManager()
    model_evalution_config=config.get_model_evaluation_config()
    model_evalution=ModelEvaluation(config=model_evalution_config)
    model_evalution.log_into_mlflow()
except Exception as e:
    raise e 


[2025-01-05 07:14:00,722: INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-01-05 07:14:00,725: INFO:common: yaml file: params.yaml loaded successfully]
[2025-01-05 07:14:00,728: INFO:common: yaml file: schema.yaml loaded successfully]
[2025-01-05 07:14:00,731: INFO:common: created directory at: artifacts]
[2025-01-05 07:14:00,733: INFO:common: created directory at: artifacts/model_evaluation]
[2025-01-05 07:14:01,331: INFO:_client: HTTP Request: GET https://dagshub.com/api/v1/repos/Immortal-Pi/ML-project-with-MLFlow "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Immortal-Pi/ML-project-with-MLFlow"

[2025-01-05 07:14:01,343: INFO:helpers: Initialized MLflow to track repo "Immortal-Pi/ML-project-with-MLFlow"]


Repository Immortal-Pi/ML-project-with-MLFlow initialized!

[2025-01-05 07:14:01,351: INFO:helpers: Repository Immortal-Pi/ML-project-with-MLFlow initialized!]
[2025-01-05 07:14:01,498: INFO:common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/01/05 07:14:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
